In [14]:
import numpy as np

Batch = 512
N = 1
d = 64

X = np.random.random(size=(1, 1, 4096))
A1 = np.random.random(size=(1, 4096, 13696))
A2 = np.random.random(size=(1, 4096, 13696))
B = np.random.random(size=(1, 13696, 4096))

def increasing_matmul_active_matmul(X, batch=64):
    last_dim = A1.shape[-1]
    for i in range(0, last_dim, batch):
        batch_A1 = A1[:, :, i:i+batch]
        batch_A2 = A2[:, :, i:i+batch]
        batch_X_a = np.matmul(X, batch_A1)
        batch_Y_a = np.matmul(X, batch_A2)
        batch_X_active = silu(batch_X_a)
        batch_R_mul = batch_X_active * batch_Y_a

        batch_B = B[:, i:i+batch, :]
        batch_res = np.matmul(batch_R_mul, batch_B)
        import pdb;pdb.set_trace()
    return acc

def silu(x):
    return x / (1 + np.exp(-x))

def matmul_active_matmul(X):
    X_a = np.matmul(X, A1)
    Y_a = np.matmul(X, A2)
    X_active = silu(X_a)

    R_mul = X_active * Y_a
    return np.matmul(R_mul, B)

# expected = softmax_then_matmul(S, V)
# actual = increasing_softmax_then_matmul(S, V)
# np.allclose(expected, actual)
incre_res = increasing_matmul_active_matmul(X)
res = matmul_active_matmul(X)

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 13696 is different from 64)

In [10]:
res.shape

(1, 1, 4096)

In [3]:
import torch
import torch.nn.functional as F

x = torch.randn((12,197,197))

In [2]:
%time softmax_then_matmul(S, V)

CPU times: user 70 µs, sys: 120 µs, total: 190 µs
Wall time: 150 µs


array([[0.69450367, 0.64144421]])

In [3]:
%time increasing_softmax_then_matmul(S, V)

CPU times: user 86 µs, sys: 146 µs, total: 232 µs
Wall time: 241 µs


array([[0.69450367, 0.64144421]])